In [1]:
!pip install datasets

     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.3 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 136 kB 40.5 MB/s 
     |████████████████████████████████| 212 kB 47.3 MB/s 
     |████████████████████████████████| 127 kB 35.7 MB/s 
     |████████████████████████████████| 144 kB 45.9 MB/s 
     |████████████████████████████████| 271 kB 49.6 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 48.9 MB/s 
     |████████████████████████████████| 895 kB 40.0 MB/s 
     |████████████████████████████████| 6.6 MB 33.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [39]:
from IPython.display import HTML, display
#Код для аккуратного вывода длинных строк
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from transformers import BertTokenizer, DistilBertForQuestionAnswering , get_scheduler, DistilBertTokenizerFast

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import copy

# Загрузка и подготовка данных

In [4]:
from datasets import load_dataset, load_metric

In [5]:
#Загрузим датасет напрямую с huggingface и разделим его на части
dataset = load_dataset("sberquad")
raw_train_dataset = dataset['train']
raw_val_dataset = dataset['validation']
raw_test_dataset = dataset['test']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

Dataset sberquad downloaded and prepared to /root/.cache/huggingface/datasets/sberquad/sberquad/1.0.0/62115d937acf2634cfacbfee10c13a7ee39df3ce345bb45af7088676f9811e77. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
raw_train_dataset.info

DatasetInfo(description='Sber Question Answering Dataset (SberQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable. Russian original analogue presented in Sberbank Data Science Journey 2017.\n', citation='@article{Efimov_2020,\n   title={SberQuAD – Russian Reading Comprehension Dataset: Description and Analysis},\n   ISBN={9783030582197},\n   ISSN={1611-3349},\n   url={http://dx.doi.org/10.1007/978-3-030-58219-7_1},\n   DOI={10.1007/978-3-030-58219-7_1},\n   journal={Experimental IR Meets Multilinguality, Multimodality, and Interaction},\n   publisher={Springer International Publishing},\n   author={Efimov, Pavel and Chertok, Andrey and Boytsov, Leonid and Braslavski, Pavel},\n   year={2020},\n   pages={3–15}\n}\n ', homepage='', license='', features={'id': Value(dtype='int32', 

In [7]:
raw_train_dataset.features

{'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
 'context': Value(dtype='string', id=None),
 'id': Value(dtype='int32', id=None),
 'question': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None)}

In [8]:
raw_train_dataset[1]

{'answers': {'answer_start': [438],
  'text': ['нитевидные водоросли, грибные нити']},
 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.',
 'id': 28101,
 'question': 'что найдено в кремнистых сланцах железорудной формации Канады?',
 'title': 'SberChallenge'}

In [9]:
raw_val_dataset[1]

{'answers': {'answer_start': [78], 'text': ['В XXVII веке до н. э.']},
 'context': 'Первые упоминания о строении человеческого тела встречаются в Древнем Египте. В XXVII веке до н. э. египетский врач Имхотеп описал некоторые органы и их функции, в частности головной мозг, деятельность сердца, распространение крови по сосудам. В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека. В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге.',
 'id': 36330,
 'question': 'Когда египетский врач Имхотеп впервые описал некоторые органы и их функции?',
 'title': 'SberChallenge'}

In [10]:
print(f"Размер тренировочного датасета {raw_train_dataset.shape}")
print(f"Размер валидационного датасета {raw_val_dataset.shape}")

Размер тренировочного датасета (45328, 5)
Размер валидационного датасета (5036, 5)


In [11]:
#Проверка, что на каждый вопрос есть один ответ
for example in raw_train_dataset:
  if len(example['answers'])!=2 or len(example['answers']['text'][0])==0:
    print(example)

In [12]:
#Будем использовать bert токенизатор для рускоязычной модели. 
#Возьмем distilbert для более быстрого обучения
PRE_TRAINED_MODEL_NAME = 'Geotrend/distilbert-base-ru-cased'
tokenizer = DistilBertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/130k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

In [14]:
# Будем использовать функции описанные в примере использования BERT для QA
# https://github.com/huggingface/transformers/blob/main/examples/pytorch/question-answering/run_qa.py

def prepare_train_features(examples):
        
        # Токенизируем примеры из датасета. Если пример длиннее, чем max_length токенов, то разобьем его на несколько. 
        tokenized_examples = tokenizer(
            examples['question'],
            examples['context'],
            truncation="only_second",
            max_length=400,
            return_offsets_mapping=True,
            return_overflowing_tokens=True,
            padding="max_length",
        )

        # Так как на один пример может приходиться несколько последовательностей, то нужно установить соответствие
        # между последовательностями токенов и примерами
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # offset mapping дает задает правило перевода позиции токена в позицию символов в изначальном примере.
        # Это нужно для вычисления start_positions и end_positions ответов на вопрос
        offset_mapping = tokenized_examples.pop("offset_mapping")

        # Будем записывать начальное и конечную позицию токенов, необходимых для ответа на вопрос
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []

        # Нам нужно вычислить start_positions и end_positions ответов на вопрос для всех примеров.
        # Для этого будем перебирать все offset_mapping
        for i, offsets in enumerate(offset_mapping):
            # Берем токены input_ids и массив sequence_ids, который задает принадлежность каждого токена последовательности.
            # В нашем случае сначала идет вопрос, а затем контекст, поэтому sequence_ids=0 это токены вопроса,
            # а sequence_ids=1 это токены контекста
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)
            sequence_ids = tokenized_examples.sequence_ids(i)

            # Если пример дал несколько последовательностей, то будем перебирать их последовательно
            sample_index = sample_mapping[i]
            answers = examples['answers'][sample_index]
            # Если ответа на вопрос нет, то будем считать, что ответом является токен CLS.
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Вычисляем индекс первого и последнего символа ответа.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                # Находим правую и левую границу токенов контекста. Контекст кодируется с помощью 1 в sequence_ids.
                token_start_index = 0
                while sequence_ids[token_start_index] != 1:
                    token_start_index += 1
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != 1:
                    token_end_index -= 1

                # Если ответ выходит за границы последовательности, то считаем, что ответа нет
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # В противном случае двигаем token_start_index и token_end_index к двум концам ответа
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)

        return tokenized_examples

In [15]:
# Токенизируем датасеты и вычисляем начальный и конечный индексы токенов, соответствующих ответу на вопрос
column_names = raw_train_dataset.column_names
encoded_train_dataset = raw_train_dataset.map(prepare_train_features, batched=True,
                                              remove_columns=column_names,
                                              desc="Running tokenizer on train dataset",)
encoded_val_dataset = raw_val_dataset.map(prepare_train_features, batched=True,
                                          remove_columns=column_names,
                                              desc="Running tokenizer on val dataset",)

Running tokenizer on train dataset:   0%|          | 0/46 [00:00<?, ?ba/s]

Running tokenizer on val dataset:   0%|          | 0/6 [00:00<?, ?ba/s]

In [16]:
from transformers import Trainer, TrainingArguments, default_data_collator
from transformers.trainer_utils import PredictionOutput

In [17]:
data_collator = default_data_collator

In [18]:
# Используем предобученную модель bert
model=DistilBertForQuestionAnswering.from_pretrained(PRE_TRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/208M [00:00<?, ?B/s]

Some weights of the model checkpoint at Geotrend/distilbert-base-ru-cased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at Geotrend/distilbert-base-ru-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should prob

In [19]:
# Параметры обучения
batch_size=32
training_args = TrainingArguments(
    PRE_TRAINED_MODEL_NAME,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [20]:
# Будем использовать класс Trainer, чтобы не писать свой цикл обучения
trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [21]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 47113
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4419


Epoch,Training Loss,Validation Loss


Saving model checkpoint to Geotrend/distilbert-base-ru-cased/checkpoint-500
Configuration saved in Geotrend/distilbert-base-ru-cased/checkpoint-500/config.json
Model weights saved in Geotrend/distilbert-base-ru-cased/checkpoint-500/pytorch_model.bin
tokenizer config file saved in Geotrend/distilbert-base-ru-cased/checkpoint-500/tokenizer_config.json
Special tokens file saved in Geotrend/distilbert-base-ru-cased/checkpoint-500/special_tokens_map.json


Epoch,Training Loss,Validation Loss
1,2.230500,1.869029
2,1.783300,1.735569
3,1.600500,1.711762


Saving model checkpoint to Geotrend/distilbert-base-ru-cased/checkpoint-1000
Configuration saved in Geotrend/distilbert-base-ru-cased/checkpoint-1000/config.json
Model weights saved in Geotrend/distilbert-base-ru-cased/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in Geotrend/distilbert-base-ru-cased/checkpoint-1000/tokenizer_config.json
Special tokens file saved in Geotrend/distilbert-base-ru-cased/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5241
  Batch size = 32
Saving model checkpoint to Geotrend/distilbert-base-ru-cased/checkpoint-1500
Configuration saved in Geotrend/distilbert-base-ru-cased/checkpoint-1500/config.json
Model weights saved in Geotrend/distilbert-base-ru-cased/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in Geotrend/distilbert-base-ru-cased/checkpoint-1500/tokenizer_config.json
Special tokens file saved in Geotrend/distilbert-base-ru-cased/checkpoint-1500/special_tokens_map.json
Saving mode

In [21]:
def prepare_validation_features(examples):
    # Функция для токенизации примеров валидационного датасета. Она отличается от prepare_train_features тем,
    # что для этого датасета мы будем сохранять только offset_mapping. Это нужно для того, чтобы мы позже могли
    # восстановить ответ по индексам токенов
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation="only_second",
        max_length=400,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Заполняем offset_mapping. Если токен не попадает в последовательность контекста, то запоминаем None
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [22]:
validation_features = raw_val_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=column_names,
    desc="Running tokenizer on val dataset"
)

Running tokenizer on val dataset:   0%|          | 0/6 [00:00<?, ?ba/s]

In [23]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5241
  Batch size = 32


In [24]:
validation_features.set_format(type=validation_features.format["type"], 
                               columns=list(validation_features.features.keys()))

In [25]:
import collections

In [26]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    # Генерация финального предсказания ответа. Будем перебирать n_best_size ответов и выберем тот, у которого самый большой score
    all_start_logits, all_end_logits = raw_predictions
    # Соответствие между примером из датасета и последовательностью токенов.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    # Сравнение количества примеров и количества последовательностей токенов. 
    # Последовательностей токенов будет больше, если какие-то примеры пришлось разбивать на несколько
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in enumerate(tqdm(examples)):
        # Индекс последовательности токенов, которая соответствует данному примеру.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Возьмем все последовательности токенов, соответствующие данному примеру.
        for feature_index in feature_indices:
            # Предсказание модели вероятности логитов.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # offset_mapping для восстановления ответа в изначальном контексте
            offset_mapping = features[feature_index]["offset_mapping"]

            # Вероятность отсутствия ответа.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Рассмотрим все варианты ответов для первых n_best_size возможных комбинаций начального и конечного логита 
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Выкинем невозможные ответы - это те ответы, у которых индексы токенов вылетели за границы или
                    # индексы не попали в область контекста
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[start_index])==0
                        or len(offset_mapping[end_index])==0
                    ):
                        continue
                    # Не рассматриваем ответы с длиной < 0, либо > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        # Выбираем лучший ответ
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # Если у нас нет ненулевых ответов, то выдаем нулевой ответ
            best_answer = {"text": "", "score": 0.0}
        
        # Выдаем лучший ответ
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [27]:
# Генерация ответов для валидационного датасета
final_predictions = postprocess_qa_predictions(raw_val_dataset, 
                                               validation_features, raw_predictions.predictions,
                                               n_best_size=10)

Post-processing 5036 example predictions split into 5241 features.


  0%|          | 0/5036 [00:00<?, ?it/s]

In [28]:
# Воспользуемся метрикой датасета SQUAD для оценки качества работы
metric = load_metric("squad")

In [37]:
# Приведем предсказания и настоящие ответы к единому формату
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in raw_val_dataset]
metric.compute(predictions=formatted_predictions, references=referencess)

{'exact_match': 55.1429706115965, 'f1': 76.10092920397281}

Обученная сеть демонстрирует f1 score 76, что очень неплохо для QA модели, но уступает качеству большой модели RuBERT от DeepPavlov, которая демонстрирует f1 score 84. Это связано с тем, что мы использовали DistilBert

In [12]:
# Подключим гугл-диск и сохраним обученную модель туда
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
trainer.save_model("drive/MyDrive/Colab Notebooks/test-squad-trained")

Saving model checkpoint to drive/MyDrive/Colab Notebooks/test-squad-trained
Configuration saved in drive/MyDrive/Colab Notebooks/test-squad-trained/config.json
Model weights saved in drive/MyDrive/Colab Notebooks/test-squad-trained/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/Colab Notebooks/test-squad-trained/tokenizer_config.json
Special tokens file saved in drive/MyDrive/Colab Notebooks/test-squad-trained/special_tokens_map.json


In [13]:
# Если нужно загрузить сохраненную на гугл диск модель, то можно воспользоваться следующим кодом
PRE_TRAINED_MODEL_NAME = 'drive/MyDrive/Colab Notebooks/test-squad-trained'
tokenizer = DistilBertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME, local_files_only=True)

In [18]:
model=DistilBertForQuestionAnswering.from_pretrained(PRE_TRAINED_MODEL_NAME, local_files_only=True)

In [43]:
# Функция для форматированного вывода контекста, вопроса и ответов
def get_answer_to_val_dataset(ind, predictions):
  context=raw_val_dataset['context'][ind]
  question=raw_val_dataset['question'][ind]
  answer=raw_val_dataset['answers'][ind]['text']
  for predicted_answer in predictions:
    if predicted_answer['id']==raw_val_dataset['id'][ind]:
      final_predicted_answer=predicted_answer['prediction_text']
  print(f'Context: {context}')
  print('-'*10)
  print(f'Question: {question}')
  print(f'True answer: {answer[0]}')
  print(f'Predicted answer: {final_predicted_answer}')

In [46]:
# Посмотрим что выдает модель для n_ind случайный примеров из валидационного датасета
n_ind=10
inds=np.random.permutation(len(raw_val_dataset))
for ind in inds[:n_ind]:
  get_answer_to_val_dataset(ind, formatted_predictions)
  print('*'*20)

Context: Ранняя история города слабо известна. Сохранившиеся источники говорят о столкновениях между греками и этрусками. Некоторое время Помпеи принадлежали Кумам, с конца VI века до н. э. находились под влиянием этрусков и входили в союз городов во главе с Капуей. При этом в 525 до н. э. был построен дорический храм в честь греческих богов. После разгрома этрусков в Ките, Сиракузах в 474 до н. э. господство в регионе вновь завоевали греки. В 20-е годы V века до н. э. вместе с другими городами Кампании были завоёваны самнитами[2]. В ходе Второй Самнитской войны самниты были разгромлены Римской республикой, а Помпеи около 310 года до н. э. стали союзниками Рима.
----------
Question: Каким народом были завоёваны Помпеи в 20-е годы V века до н. э.?
True answer: самнитами
Predicted answer: самнитами
********************
Context: Аэропорт Байкал — международный аэропорт города Улан-Удэ. Расположен в пределах городского округа Улан-Удэ, в 15 км западнее от центра города, и в 75 км к юго-вос

Видим, что на большинство вопросов модель дает правильный ответ